In [1]:
import cv2
import numpy as np

### 取得圖像

In [3]:
image = cv2.imread("../images/lennaGrayScaleCrop.jpg",0)
height = image.shape[0]
width = image.shape[1]
print("height: "+str(height))
print("width: "+str(width))

height: 480
width: 640


### 獲取要隱藏的訊息

1. 逐字讀取原始訊息
2. 轉換為ascii碼
3. ascii碼轉換為二進制(0bxxxxxxx)的字串
4. replace()替換0b為空
5. 七位數二進制填充為八位數

In [4]:
codewords = [[0 for j in range(width)] for i in range(height)]
received = [[0 for j in range(width)] for i in range(height)]

In [5]:
def bitstouint(bits):
    uint = 0
    for bit in bits:
        uint = (uint <<1) | bit
    return uint

In [6]:
def encodeHamming(msg,n,k) :
    m = n - k
    encoded = []
    r = 0 #rth redundant bit
    
    #set up codeword loop
    for i in range(1, n+1):
        if i == 2 ** r :
            encoded.append(0)
            r += 1
        else :
            encoded.append(int(msg[i-r-1]))

    #set paritybit values
    rval = 0
    for i in range(m) :
        rval = 2 ** i
        count1 = 0
        for j in range(1,n+1) :
            if ((j & (1 << i)) & rval == rval and encoded[j-1] ==1):
                count1 += 1
            if (count1 % 2 == 1) :
                encoded[rval-1] = 1
    return encoded

In [7]:
for i in range(height):
 for j in range(width):
   codewords[i][j] =encodeHamming(format(image[i][j],'08b'), 12,8)

In [8]:
def Hamming(encoded):
    error = 0
    idx = 1
    for i in encoded :
        if (i==1) :
            error = error ^ idx
        idx += 1
    if (error > 0 and error < idx) :
        encoded[error-1] = int(not encoded[error-1])
    r = 0
    bits = []
    for i in range(1,len(encoded)+1):
        if (i == 2 ** r) :
            r += 1
        else :
            bits.append(encoded[i-1])
    return bits

In [9]:
#read and correct every pixel
for i in range(height):
    for j in range(width):
        received[i][j] = bitstouint(Hamming(codewords[i][j]))
received = np.array(received,dtype=np.uint8)
cv2.imwrite("../images/LennaAfterHamming74.jpg", received)

True

In [10]:
def Generators (n, k):
    m = n-k
    G = [[0 for j in range(n)] for i in range(k)]
    H = [[0 for j in range(n)] for i in range(m)]
    for i in range (k):
        G[i][i] = 1
    for i in range (m):
        H[i][k+i] = 1
    r = 0
    for i in range (1,n+1):
        if i == 2**r :
            r += 1
        else :
            bin = format(i,'0'+str(m)+ 'b')
            for j in range(1,m+1):
                G[i-r-1][k+j-1] = int(bin[-j])
                H[j-1][i-r-1] = int(bin[-j])
    return G, H

In [11]:
def encodeHammingM(msg,G):
    k = len(G)
    n = len(G[0])
    codeword = [0 for i in range(n)]

    #Matrix Xor Multiplication
    for i in range(n):
        for j in range(k):
            codeword[i] = codeword[i] ^ (int(msg[j]) & G[j][i])
    return codeword

In [12]:
def decodeHammingM(codeword, H):
    n = len(codeword)
    m = len(H)
    syndrome = [0 for i in range(m)]

    #Matrix Xor multiplication
    for i in range(m):
        for j in range(n):
            syndrome[i] = syndrome[i] ^ (H[i][j] & int(codeword[j]) )

    #Check any syndrome columns (error correction)
    for column in range(n):
        allsame = True
        for row in range(m):
            if (H[row][column] != syndrome[row]) :
                allsame = False
        if allsame :
            codeword[column] = int(not codeword[column])
            break
    return codeword